## Customize Your Lineup - Trial Code
Get the python/pandas logic working for the customize page before incorporating it into the flask app/js/html.

In [98]:
import pandas as pd
import numpy as np
from scipy import stats

# #try using plotly for graphing - should help transition to later javascript graphs
# import plotly
# import plotly.plotly as py
# import plotly.figure_factory as ff
# import plotly.graph_objs as go
# #need to use Plotly API Key in order to get it to run
# plotly.tools.set_credentials_file(username='mtsuomi', api_key='KU6JDtNccJICaVmeF2gF')

from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.automap import automap_base
import json

In [99]:
#create engine to the database
db = create_engine('sqlite:///../fantasy_football_2018.db')

#create inspector so can see schema of database visually
inspector = inspect(db)

#Database Schema - verify the database has the info we are expecting by printing out all the schema visually
for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print(table_name, column.get('name'), column.get('type'))
    print()

#other options for using SQLAlchemy sort of coding instead of traditional SQL queries
# engine = create_engine('sqlite:///raw_data/hawaii.sqlite')
# Base = automap_base()
# Base.prepare(engine, reflect=True)
# session = Session(bind=engine)
# inspector = inspect(engine)

def_scores Unnamed: 0 BIGINT
def_scores PLAYER, TEAM POS TEXT
def_scores team_abbr TEXT
def_scores week_2_opp TEXT
def_scores week_2_proj FLOAT
def_scores week_2_actual BIGINT
def_scores week_3_opp TEXT
def_scores week_3_proj FLOAT
def_scores binned TEXT
def_scores score_sentiment TEXT

week1_ppr_actuals PLAYER TEXT
week1_ppr_actuals POS TEXT
week1_ppr_actuals TEAM TEXT
week1_ppr_actuals FPTS_PPR_ACTUAL FLOAT
week1_ppr_actuals WEEK BIGINT
week1_ppr_actuals PK INTEGER

week1_ppr_projections PLAYER TEXT
week1_ppr_projections POS TEXT
week1_ppr_projections TEAM TEXT
week1_ppr_projections FPTS_PPR_ESPN FLOAT
week1_ppr_projections FPTS_PPR_CBS FLOAT
week1_ppr_projections FPTS_PPR_SHARKS FLOAT
week1_ppr_projections FPTS_PPR_SCOUT FLOAT
week1_ppr_projections FPTS_PPR_PRVS_WK_ACTUAL FLOAT
week1_ppr_projections PK INTEGER

week1_ppr_projections_all_scraped PLAYER TEXT
week1_ppr_projections_all_scraped POS TEXT
week1_ppr_projections_all_scraped TEAM TEXT
week1_ppr_projections_all_scraped FPTS_PP

##### Customize Projection Average

In [64]:
week = 3
espn_wt = 15
cbs_wt = 40
sharks_wt = 15
scout_wt = 0
prior_wt = 30
def_boost_wt = 'Full'
overunder_boost_wt = 'Partial'
twitter_boost_wt = 'None'

#convert weights to percentages
espn_pct = espn_wt/100
cbs_pct = cbs_wt/100
sharks_pct = sharks_wt/100
scout_pct = scout_wt/100
prior_pct = prior_wt/100

In [28]:
#get the projections for your week selected
stmt = f"""
        SELECT *
        FROM week{week}_ppr_projections
       """

#need to input db.session.bind in flask app due to different initalization process
df_proj = pd.read_sql_query(stmt, db)
print(df_proj.shape)
df_proj.head()

(357, 9)


,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5


In [29]:
df_proj.columns

Index(['PLAYER', 'POS', 'TEAM', 'FPTS_PPR_ESPN', 'FPTS_PPR_CBS',
       'FPTS_PPR_SHARKS', 'FPTS_PPR_SCOUT', 'FPTS_PPR_PRVS_WK_ACTUAL', 'PK'],
      dtype='object')

In [31]:
#create initial custom weighted average of the five projection sources based on user input
df_proj['FPTS_PPR_CUSTOM_AVG'] = df_proj.apply(lambda row: 
                                    (row['FPTS_PPR_ESPN']*espn_pct +
                                    row['FPTS_PPR_CBS']*cbs_pct +
                                    row['FPTS_PPR_SHARKS']*sharks_pct +
                                    row['FPTS_PPR_SCOUT']*scout_pct +
                                    row['FPTS_PPR_PRVS_WK_ACTUAL']*prior_pct),
                                    axis='columns')
df_proj.head()

,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375


##### Boost Factor Logic - Import Data (later DB connection), Merge Data, then Do Logic

In [113]:
# ### FIGURE OUT UPDATED DB CONNECTION LATER ###
# #import the data for now
# df_opp_defense = pd.read_excel('../def_scoring_proj.xlsx')
# df_overunder = pd.read_csv('../week_2_over_unders.csv')
# df_tweets = pd.read_csv('../expert_tweets_week_3.csv')

In [115]:
#get the defense data for your week selected - all defense data for each week is one database,
#select only columns you want
#NOTE - currently it is pulling in defense sentiment always for week 3 - no sentiments for other weeks
stmt_def = f"""
        SELECT team_abbr, week_{week}_opp, week_{week}_proj, score_sentiment
        FROM def_scores
       """

#need to input db.session.bind in flask app due to different initalization process
df_defense = pd.read_sql_query(stmt_def, db)

#currently score_sentiment is for the offensive opponenet perspective so Min's really good Defense
#against Buf has score sentiment of 'Terrible' which is terrible for Buf, so all Buf players
#would get negative points weighting - rename column
#rename team_abbr to def_team_abbr
df_defense.rename(columns={'score_sentiment': 'offense_opp_def_impact',
                           'team_abbr': 'def_team_abbr'}, inplace=True)

print(df_defense.shape)
df_defense.head()

(32, 4)


,def_team_abbr,week_3_opp,week_3_proj,offense_opp_def_impact
0,NYJ,Cle,5.2,Neutral
1,Min,Buf,13.2,Terrible
2,Bal,Den,6.7,Neutral
3,LAR,LAC,6.8,Neutral
4,Chi,Ari,6.5,Neutral


In [105]:
#get the overunder data for your week selected
#select only columns you want
stmt_overunder = f"""
        SELECT *
        FROM week_{week}_over_unders
       """

#need to input db.session.bind in flask app due to different initalization process
df_overunder = pd.read_sql_query(stmt_overunder, db)
print(df_overunder.shape)
df_overunder.head()

(32, 6)


,Unnamed: 0,team,over_under,team_abbr,binned,score_sentiment
0,0,Arizona Cardinals,37.5,Ari,"(37.0, 40.146]",Terrible
1,1,Atlanta Falcons,53.5,Atl,"(52.479, 56.5]",Awesome
2,2,Baltimore Ravens,43.5,Bal,"(42.792, 48.958]",Neutral
3,3,Buffalo Bills,41.0,Buf,"(40.146, 42.792]",Concerned
4,4,Carolina Panthers,43.5,Car,"(42.792, 48.958]",Neutral


In [107]:
#get the tweet data for your week selected
#select only columns you want
stmt_tweets = f"""
        SELECT *
        FROM week_{week}_tweets
       """

#need to input db.session.bind in flask app due to different initalization process
df_tweets = pd.read_sql_query(stmt_tweets, db)
print(df_tweets.shape)
df_tweets.head()

(200, 8)


,Unnamed: 0,number,player,compound_score,binned,expert_sentiment,color,all_tweets
0,0,1,Cam Newton,-0.223550,"(-0.5, -0.02]",Concerned,yellow,['Matt Ryan did his best Cam Newton impression...
1,1,2,Drew Brees,0.168067,"(0.02, 0.5]",Good,blue,['Drew Brees leads comeback win over Browns ht...
2,2,3,Aaron Rodgers,0.062883,"(0.02, 0.5]",Good,blue,['Aaron Rodgers escapes without further damage...
3,3,4,Alex Smith,0.085200,"(0.02, 0.5]",Good,blue,['RT @JeffRatcliffe: QBs without any end zone ...
4,4,5,Ben Roethlisberger,0.041513,"(0.02, 0.5]",Good,blue,['Ben Roethlisberger back at practice Friday h...


In [112]:
# #Connor's code for binning the over-unders
# high = df_overunder['over_under'].max()
# print(high)
# low = df_overunder['over_under'].min()
# print(low)
# mean = df_overunder['over_under'].mean()
# print(mean)

# #create bins from range of over unders
# bins = [(low-.5), ((low+low+mean)/3), ((low+mean+mean)/3), ((high+mean+mean)/3), ((high+high+mean)/3), (high+.5)]
# df_overunder['binned'] = pd.cut(df_overunder['over_under'], bins)
# df_overunder.head()

# #convert binned values to string so we can categorize with adjectives
# df_overunder['binned'] = df_overunder['binned'].map(lambda x: str(x))

# #create adjectives from bins
# score_bins = {
#     '(42.0, 43.521]': 'Terrible',
#     '(43.521, 44.542]': 'Concerned',
#     '(44.542, 48.208]': 'Neutral',
#     '(48.208, 50.854]': 'Good',
#     '(50.854, 54.0]': 'Awesome'
# }

# df_overunder['score_sentiment'] = df_overunder['binned'].map(score_bins)

# color_bins = {
#     'Terrible': 'red',
#     'Concerned': 'yellow',
#     'Neutral': 'grey',
#     'Good': 'blue',
#     'Awesome': 'green'
# }

# df_overunder['color'] = df_overunder['score_sentiment'].map(color_bins)
# df_overunder

In [116]:
#merge on the boost factor data to players
#data: opposing defense, opposing defense bin, twitter sentiment, twitter sentiment bin, vegas overunder, vegas overunder bin

#check size of starting df
print(df_proj.shape)

#first merge in the twitter sentiment - always do left merge (rather have Nans for boost factors than lose players)
df_boosts = pd.merge(df_proj, df_tweets[['player', 'compound_score', 'expert_sentiment']],
                     how='left', left_on='PLAYER', right_on='player')
df_boosts.drop('player', axis='columns', inplace=True) #don't need to keep this added player name column
print(df_boosts.shape)

#second merge in the overunder - always do left merge (rather have Nans for boost factors than lose players)
df_boosts = pd.merge(df_boosts, df_overunder[['team_abbr', 'over_under', 'score_sentiment']],
                     how='left', left_on='TEAM', right_on='team_abbr')
df_boosts.drop('team_abbr', axis='columns', inplace=True) #don't need to keep this added team abbr column
print(df_boosts.shape)

#third merge in the defense projections - always do left merge (rather have Nans for boost factors than lose players)
#will merge by the team of the datframe and in defense looking for the opponent (that is offensive opponent of the defense listed as team)
df_boosts = pd.merge(df_boosts, df_defense[['def_team_abbr', f"week_{week}_opp", f"week_{week}_proj", 'offense_opp_def_impact']],
                     how='left', left_on='TEAM', right_on=f"week_{week}_opp")
df_boosts.drop(f"week_{week}_opp", axis='columns', inplace=True) #don't need to keep this added team abbr column
print(df_boosts.shape)

#convert any of the Nans in our binned sentiment columns to Neutral (that will just apply a zero boost to those players)
df_boosts.fillna('Neutral', inplace=True)

df_boosts.head(30)

(357, 10)
(357, 12)
(357, 14)
(357, 17)


,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG,compound_score,expert_sentiment,over_under,score_sentiment,def_team_abbr,week_3_proj,offense_opp_def_impact
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160,-0.09417,Concerned,53.5,Awesome,Atl,4.1,Good
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165,0.296,Good,41.5,Concerned,Sea,4.7,Neutral
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795,0.00398333,Neutral,52,Good,Det,1.5,Awesome
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160,0.123336,Good,48,Neutral,LAC,4.2,Good
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375,-0.046625,Concerned,53.5,Awesome,TB,3.6,Good
5,Ryan Fitzpatrick,QB,TB,19.7,20.0,20.5,25.0,30.0,6,23.030,0.0424545,Good,53.5,Awesome,Pit,5.5,Neutral
6,Patrick Mahomes,QB,KC,19.5,23.0,22.4,28.0,38.8,7,27.125,0.0619618,Good,56,Awesome,SF,4.6,Neutral
7,David Johnson,RB,Ari,19.4,15.0,16.0,17.9,6.1,8,13.140,0.0807625,Good,37.5,Terrible,Chi,6.5,Neutral
8,Michael Thomas,WR,NO,19.4,21.0,21.8,22.3,30.9,9,23.850,0.0767,Good,53.5,Awesome,Atl,4.1,Good
9,Aaron Rodgers,QB,GB,19.3,24.0,21.2,25.3,16.0,10,20.475,0.062883,Good,45.5,Neutral,Wsh,6.1,Neutral


In [62]:
# #apply defensive boost factor
# # def_boost = 'Full'
# # overunder_boost = 'Partial'
# # twitter_boost = 'None'

# player = 'Alvin Kamara'
# team = 'NO'
# custom_points = 20.16
# expert_sentiment = 'Concerned'
# score_sentiment = 'Good'
# defense_sentiment = 'Terrible'

In [117]:
def over_under_boost_calculator(row, user_boost_weight):
    #define the user's boost multiplier based on their selection of how much of the factor to apply
    if user_boost_weight == 'Full':
        user_boost_multiplier = 1.0
    elif user_boost_weight == 'Partial':
        user_boost_multiplier = 0.5
    else:
        user_boost_multiplier = 0
        
    #these are our custom boost factors based on our o/u score sentiments (can change these on the backend however we want)
    over_under_boost_values = {'Awesome': 0.2, 'Good': 0.1, 'Neutral': 0, 'Concerned': -0.1, 'Terrible': -0.2}
    
    #get the starting custom weighted average fantasy points projections by the user
    custom_points = row['FPTS_PPR_CUSTOM_AVG']
    
    #get boost multiplier based on the overunder binned sentiments and our dictionary of boost factors
    boost_multiplier = over_under_boost_values.get(row['score_sentiment'], 0)
    
    #calculate the boost points
    over_under_boost_points = custom_points * boost_multiplier * user_boost_multiplier
    
    return over_under_boost_points

In [118]:
def twitter_boost_calculator(row, user_boost_weight):
    #define the user's boost multiplier based on their selection of how much of the factor to apply
    if user_boost_weight == 'Full':
        user_boost_multiplier = 1.0
    elif user_boost_weight == 'Partial':
        user_boost_multiplier = 0.5
    else:
        user_boost_multiplier = 0
        
    #these are our custom boost factors based on our twitter sentiments (can change these on the backend however we want)
    twitter_boost_values = {'Awesome': 0.2, 'Good': 0.1, 'Neutral': 0, 'Concerned': -0.1, 'Terrible': -0.2}
    
    #get the starting custom weighted average fantasy points projections by the user
    custom_points = row['FPTS_PPR_CUSTOM_AVG']
    
    #get boost multiplier based on the overunder binned sentiments and our dictionary of boost factors
    boost_multiplier = twitter_boost_values.get(row['expert_sentiment'], 0)
    
    #calculate the boost points
    twitter_boost_points = custom_points * boost_multiplier * user_boost_multiplier
    
    return twitter_boost_points

In [119]:
def defense_boost_calculator(row, user_boost_weight):
    #if the player looking it as a Defense, we will want to skip and not boost the defense projection
    if row['POS'] == 'D/ST':
        return 0.0
    
    #otherwise apply the defense boost to all other positions (may in future want to consider removing kickers as
    #they are often unpredictable, and maybe the correlation of a harder defense is even positive on kicker points b/c they kick more FGs than TDs)
    else:
        #define the user's boost multiplier based on their selection of how much of the factor to apply
        if user_boost_weight == 'Full':
            user_boost_multiplier = 1.0
        elif user_boost_weight == 'Partial':
            user_boost_multiplier = 0.5
        else:
            user_boost_multiplier = 0

        #these are our custom boost factors based on our opposing defenses(can change these on the backend however we want)
        #the sentiment stored is from the offensive perspective of the opp def impact, so 'Good' means they 
        #are facing an easy defense and will get a positive boost, 'Terrible' means they are facing a hard 
        #defense and will get negative boost
        opp_defense_boost_values = {'Awesome': 0.2, 'Good': 0.1, 'Neutral': 0, 'Concerned': -0.1, 'Terrible': -0.2}

        #get the starting custom weighted average fantasy points projections by the user
        custom_points = row['FPTS_PPR_CUSTOM_AVG']

        #get boost multiplier based on the overunder binned sentiments and our dictionary of boost factors
        boost_multiplier = opp_defense_boost_values.get(row['offense_opp_def_impact'], 0)

        #calculate the boost points
        defense_boost_points = custom_points * boost_multiplier * user_boost_multiplier

        return defense_boost_points

In [120]:
print(overunder_boost_wt)
print(twitter_boost_wt)
print(def_boost_wt)

Partial
None
Full


In [121]:
df_boosts['OVER_UNDER_BOOST'] = df_boosts.apply(lambda row: 
                                    over_under_boost_calculator(row, overunder_boost_wt),
                                    axis='columns')

In [122]:
df_boosts['TWITTER_BOOST'] = df_boosts.apply(lambda row: 
                                    twitter_boost_calculator(row, twitter_boost_wt),
                                    axis='columns')

In [123]:
df_boosts['OPPOSING_DEFENSE_BOOST'] = df_boosts.apply(lambda row: 
                                        defense_boost_calculator(row, def_boost_wt),
                                        axis='columns')

In [124]:
df_boosts.head()

,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG,compound_score,expert_sentiment,over_under,score_sentiment,def_team_abbr,week_3_proj,offense_opp_def_impact,OVER_UNDER_BOOST,TWITTER_BOOST,OPPOSING_DEFENSE_BOOST
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160,-0.09417,Concerned,53.5,Awesome,Atl,4.1,Good,2.01600,-0.0,2.0160
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165,0.296,Good,41.5,Concerned,Sea,4.7,Neutral,-0.90825,0.0,0.0000
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795,0.00398333,Neutral,52,Good,Det,1.5,Awesome,1.03975,0.0,4.1590
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160,0.123336,Good,48,Neutral,LAC,4.2,Good,0.00000,0.0,2.5160
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375,-0.046625,Concerned,53.5,Awesome,TB,3.6,Good,2.03750,-0.0,2.0375


In [130]:
df_boosts.columns

Index(['PLAYER', 'POS', 'TEAM', 'FPTS_PPR_ESPN', 'FPTS_PPR_CBS',
       'FPTS_PPR_SHARKS', 'FPTS_PPR_SCOUT', 'FPTS_PPR_PRVS_WK_ACTUAL', 'PK',
       'FPTS_PPR_CUSTOM_AVG', 'compound_score', 'expert_sentiment',
       'over_under', 'score_sentiment', 'def_team_abbr', 'week_3_proj',
       'offense_opp_def_impact', 'OVER_UNDER_BOOST', 'TWITTER_BOOST',
       'OPPOSING_DEFENSE_BOOST', 'FPTS_PPR_CUSTOM_AVG_BOOST'],
      dtype='object')

In [128]:
#calculate final FPTS ADDING UP ALL BOOST PTS TO THE CUSTOM AVG

df_boosts['FPTS_PPR_CUSTOM_AVG_BOOST'] = df_boosts[['FPTS_PPR_CUSTOM_AVG', 'OPPOSING_DEFENSE_BOOST',
                                                    'OVER_UNDER_BOOST', 'TWITTER_BOOST']].sum(axis='columns') 

In [129]:
df_boosts

,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG,...,expert_sentiment,over_under,score_sentiment,def_team_abbr,week_3_proj,offense_opp_def_impact,OVER_UNDER_BOOST,TWITTER_BOOST,OPPOSING_DEFENSE_BOOST,FPTS_PPR_CUSTOM_AVG_BOOST
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160,...,Concerned,53.5,Awesome,Atl,4.1,Good,2.01600,-0.0,2.0160,24.19200
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165,...,Good,41.5,Concerned,Sea,4.7,Neutral,-0.90825,0.0,0.0000,17.25675
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795,...,Neutral,52,Good,Det,1.5,Awesome,1.03975,0.0,4.1590,25.99375
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160,...,Good,48,Neutral,LAC,4.2,Good,0.00000,0.0,2.5160,27.67600
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375,...,Concerned,53.5,Awesome,TB,3.6,Good,2.03750,-0.0,2.0375,24.45000
5,Ryan Fitzpatrick,QB,TB,19.7,20.0,20.5,25.0,30.0,6,23.030,...,Good,53.5,Awesome,Pit,5.5,Neutral,2.30300,0.0,0.0000,25.33300
6,Patrick Mahomes,QB,KC,19.5,23.0,22.4,28.0,38.8,7,27.125,...,Good,56,Awesome,SF,4.6,Neutral,2.71250,0.0,0.0000,29.83750
7,David Johnson,RB,Ari,19.4,15.0,16.0,17.9,6.1,8,13.140,...,Good,37.5,Terrible,Chi,6.5,Neutral,-1.31400,0.0,0.0000,11.82600
8,Michael Thomas,WR,NO,19.4,21.0,21.8,22.3,30.9,9,23.850,...,Good,53.5,Awesome,Atl,4.1,Good,2.38500,0.0,2.3850,28.62000
9,Aaron Rodgers,QB,GB,19.3,24.0,21.2,25.3,16.0,10,20.475,...,Good,45.5,Neutral,Wsh,6.1,Neutral,0.00000,0.0,0.0000,20.47500


In [131]:
#reorder dataframe and limits columns for the final output data and sort descending by custom poitns
df_boosts_output = df_boosts[['PLAYER', 'POS', 'TEAM', 'FPTS_PPR_CUSTOM_AVG_BOOST',
                              'FPTS_PPR_ESPN', 'FPTS_PPR_CBS', 'FPTS_PPR_SHARKS', 'FPTS_PPR_SCOUT',
                              'FPTS_PPR_PRVS_WK_ACTUAL', 'FPTS_PPR_CUSTOM_AVG', 
                              'OVER_UNDER_BOOST', 'TWITTER_BOOST',
                              'OPPOSING_DEFENSE_BOOST']].sort_values('FPTS_PPR_CUSTOM_AVG_BOOST',
                                                                    ascending=False)

In [132]:
df_boosts_output

,PLAYER,POS,TEAM,FPTS_PPR_CUSTOM_AVG_BOOST,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,FPTS_PPR_CUSTOM_AVG,OVER_UNDER_BOOST,TWITTER_BOOST,OPPOSING_DEFENSE_BOOST
19,Ben Roethlisberger,QB,Pit,31.72800,18.0,22.0,21.6,26.0,39.0,26.440,2.64400,0.0,2.6440
6,Patrick Mahomes,QB,KC,29.83750,19.5,23.0,22.4,28.0,38.8,27.125,2.71250,0.0,0.0000
8,Michael Thomas,WR,NO,28.62000,19.4,21.0,21.8,22.3,30.9,23.850,2.38500,0.0,2.3850
3,Todd Gurley,RB,LAR,27.67600,20.9,23.0,20.9,30.5,32.3,25.160,0.00000,0.0,2.5160
28,Matt Ryan,QB,Atl,27.54600,17.2,20.0,21.1,26.7,30.7,22.955,2.29550,0.0,2.2955
2,Tom Brady,QB,NE,25.99375,20.9,23.0,23.6,26.4,16.4,20.795,1.03975,0.0,4.1590
5,Ryan Fitzpatrick,QB,TB,25.33300,19.7,20.0,20.5,25.0,30.0,23.030,2.30300,0.0,0.0000
52,JuJu Smith-Schuster,WR,Pit,25.30800,14.5,18.0,15.9,22.7,31.1,21.090,2.10900,-0.0,2.1090
4,Antonio Brown,WR,Pit,24.45000,20.2,23.0,22.9,25.1,15.7,20.375,2.03750,-0.0,2.0375
0,Alvin Kamara,RB,NO,24.19200,21.8,21.0,20.8,30.4,17.9,20.160,2.01600,-0.0,2.0160


In [137]:
df_tweets.columns

Index(['Unnamed: 0', 'number', 'player', 'compound_score', 'binned',
       'expert_sentiment', 'color', 'all_tweets'],
      dtype='object')

In [134]:
df_overunder2 = pd.DataFrame(columns=['Unnamed: 0', 'team', 'over_under', 'team_abbr', 'binned',
       'score_sentiment'])

In [135]:
df_overunder2

,Unnamed: 0,team,over_under,team_abbr,binned,score_sentiment


##### Converting Our DF to a JSON File
##### !!!Don't use .to_json it makes it all a string, better way is use .to_dict('records')

Note: when you return the json in flask, you'll also want to jsonify it to make sure it is formatted well.  return jsonify(json)

In [12]:
#create json that is a list of dictionaries, where each row is its own dictionary with column:value pairs
df_week1_ppr_projections.head(10).to_json('filename_test', orient='records')


In [56]:
json_test = df_week1_ppr_projections.head(10).to_json(orient='records')
jsonfiles = json.loads(json_test)

jsonfiles

[{'PLAYER': 'Alvin Kamara',
  'POS': 'RB',
  'TEAM': 'NO',
  'FPTS_PPR_ESPN': 22.2,
  'FPTS_PPR_CBS': 19.0,
  'FPTS_PPR_SHARKS': 19.1,
  'FPTS_PPR_SCOUT': 34.0,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 1},
 {'PLAYER': 'David Johnson',
  'POS': 'RB',
  'TEAM': 'Ari',
  'FPTS_PPR_ESPN': 21.3,
  'FPTS_PPR_CBS': 19.0,
  'FPTS_PPR_SHARKS': 23.6,
  'FPTS_PPR_SCOUT': 27.0,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 2},
 {'PLAYER': 'Todd Gurley',
  'POS': 'RB',
  'TEAM': 'LAR',
  'FPTS_PPR_ESPN': 21.2,
  'FPTS_PPR_CBS': 23.0,
  'FPTS_PPR_SHARKS': 19.3,
  'FPTS_PPR_SCOUT': 28.5,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 3},
 {'PLAYER': 'Antonio Brown',
  'POS': 'WR',
  'TEAM': 'Pit',
  'FPTS_PPR_ESPN': 19.5,
  'FPTS_PPR_CBS': 22.0,
  'FPTS_PPR_SHARKS': 19.3,
  'FPTS_PPR_SCOUT': 17.3,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 4},
 {'PLAYER': 'Tom Brady',
  'POS': 'QB',
  'TEAM': 'NE',
  'FPTS_PPR_ESPN': 19.4,
  'FPTS_PPR_CBS': 20.0,
  'FPTS_PPR_SHARKS': 27.4,
  'FPTS_PPR_SCOUT': 23.3,
  'FPTS_PP

In [51]:
jsonify(json)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.